In [ ]:
### Connect with the dataset in the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Set up

In [ ]:
import pandas as pd
import numpy as np
import re

# Load the dataset , in label BA = 1, DA =2, DS = 3

In [ ]:
ba = pd.read_csv('/content/drive/MyDrive/INFO 5100/INFO 5100 Project 3/Dataset/BusinessAnalyst.csv')

In [ ]:
# add the label column
ba['Position'] = 1

In [ ]:
ba.head()

,Unnamed: 0,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,Position
0,0,0,Business Analyst - Clinical & Logistics Platform,$56K-$102K (Glassdoor est.),Company Overview\n\n\nAt Memorial Sloan Ketter...,3.9,Memorial Sloan-Kettering\n3.9,"New York, NY","New York, NY",10000+ employees,1884,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An...",-1,1
1,1,1,Business Analyst,$56K-$102K (Glassdoor est.),We are seeking for an energetic and collaborat...,3.8,Paine Schwartz Partners\n3.8,"New York, NY","New York, NY",1 to 50 employees,-1,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1,TRUE,1
2,2,2,Data Analyst,$56K-$102K (Glassdoor est.),"For more than a decade, Asembia has been worki...",3.6,Asembia\n3.6,"Florham Park, NJ","Florham Park, NJ",501 to 1000 employees,2004,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$5 to $10 million (USD),-1,-1,1
3,3,3,"Information Security Analyst, Incident Response",$56K-$102K (Glassdoor est.),Job Description Summary\nThe Information Secur...,3.6,BD\n3.6,"Franklin Lakes, NJ","Franklin Lakes, NJ",10000+ employees,1897,Company - Public,Health Care Products Manufacturing,Manufacturing,$10+ billion (USD),"Abbott, Siemens, Baxter",-1,1
4,4,4,Analyst - FP&A Global Revenue,$56K-$102K (Glassdoor est.),Magnite is the world's largest independent sel...,3.4,Rubicon Project\n3.4,"New York, NY","Los Angeles, CA",201 to 500 employees,2007,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"PubMatic, AppNexus, Index Exchange",TRUE,1


In [ ]:
da = pd.read_csv('/content/drive/MyDrive/INFO 5100/INFO 5100 Project 3/Dataset/DataAnalyst.csv')

In [ ]:
# add the label column
da['Position'] = 2

In [ ]:
da.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,Position
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True,2
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1,2
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1,2
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,-1,2
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True,2


In [ ]:
ds = pd.read_csv('/content/drive/MyDrive/INFO 5100/INFO 5100 Project 3/Dataset/DataScientist.csv')

In [ ]:
# add the label column
ds['Position'] = 3

In [ ]:
ds.head()

,Unnamed: 0,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,Position
0,0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1,3
1,1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1,3
2,2,2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True,3
3,3,3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1,3
4,4,4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1,3


#Data Pre-processing

In [ ]:
print("ba:", ba.isnull().sum())
print("----")
print("da:" ,da.isnull().sum())
print("----")
print("ds:" ,ds.isnull().sum())

ba: Unnamed: 0           0
index                0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         0
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
Competitors          0
Easy Apply           0
Position             0
dtype: int64
----
da: Unnamed: 0           0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         1
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
Competitors          0
Easy Apply           0
Position             0
dtype: int64
----
ds: Unnamed: 0           0
index                0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company N

In [ ]:
### Needed column names
col_names = ["Job Title",
"Salary Estimate",
"Job Description",
"Rating",
"Company Name",
"Location",
"Headquarters",
"Size",
"Founded",
"Type of ownership",
"Industry",
"Sector",
"Revenue",
"Competitors",
"Easy Apply",
"Position",]

In [ ]:
df = ba.loc[: , col_names].append(da.loc[: , col_names])
df = df.loc[:, col_names].append(ds.loc[: , col_names])

In [ ]:
df.reset_index(inplace = True ,drop= True)

In [ ]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,Position
0,Business Analyst - Clinical & Logistics Platform,$56K-$102K (Glassdoor est.),Company Overview\n\n\nAt Memorial Sloan Ketter...,3.9,Memorial Sloan-Kettering\n3.9,"New York, NY","New York, NY",10000+ employees,1884,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An...",-1,1
1,Business Analyst,$56K-$102K (Glassdoor est.),We are seeking for an energetic and collaborat...,3.8,Paine Schwartz Partners\n3.8,"New York, NY","New York, NY",1 to 50 employees,-1,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1,TRUE,1
2,Data Analyst,$56K-$102K (Glassdoor est.),"For more than a decade, Asembia has been worki...",3.6,Asembia\n3.6,"Florham Park, NJ","Florham Park, NJ",501 to 1000 employees,2004,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$5 to $10 million (USD),-1,-1,1
3,"Information Security Analyst, Incident Response",$56K-$102K (Glassdoor est.),Job Description Summary\nThe Information Secur...,3.6,BD\n3.6,"Franklin Lakes, NJ","Franklin Lakes, NJ",10000+ employees,1897,Company - Public,Health Care Products Manufacturing,Manufacturing,$10+ billion (USD),"Abbott, Siemens, Baxter",-1,1
4,Analyst - FP&A Global Revenue,$56K-$102K (Glassdoor est.),Magnite is the world's largest independent sel...,3.4,Rubicon Project\n3.4,"New York, NY","Los Angeles, CA",201 to 500 employees,2007,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"PubMatic, AppNexus, Index Exchange",TRUE,1


In [ ]:
### add the Hourly_pay in the dataset
df["Hourly_pay"] = 0

In [ ]:
### set the Hourly pay as 1
df.loc[df.loc[df['Salary Estimate'].str.contains('Per') == True].index, "Hourly_pay"] = 1

In [ ]:
jj = df['Salary Estimate'].str.split('-', n=1, expand=True)
tt = jj[0].str.replace('$', "")
tt = tt.str.replace('K', '000')
df['lower_salary'] = tt
tt = jj[1].str.replace('$', "")
tt = tt.str.replace('K\D+', '000', regex=True)
tt = tt.str.extract(r'(\d+)')
df['higher_salary'] = tt



In [ ]:
### same issue in Hourly Pay
df.loc[df['Hourly_pay'] == 1]

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,Position,Hourly_pay,lower_salary,higher_salary
7030,"Clinical Laboratory Scientist, Per Diem",$34-$53 Per Hour(Glassdoor est.),"As the Clinical Laboratory Scientist, you will...",3.8,UC Davis Medical Center\n3.8,"Los Angeles, CA","Sacramento, CA",10000+ employees,-1,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1,3,1,34,53
7031,Sr. Data Analyst,$34-$53 Per Hour(Glassdoor est.),"As a Sr. Data Analyst, you will work closely w...",2.8,SAG-AFTRA Health Plan and SAG-Producers Pensio...,"Burbank, CA","Burbank, CA",Unknown,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1,-1,3,1,34,53
7032,"Lead Data Science Instructor, Data Scientist",$34-$53 Per Hour(Glassdoor est.),Job Description\nGalvanize is a dynamic learni...,-1.0,Galvanize Inc.,"Los Angeles, CA","Harrisburg, NC",51 to 200 employees,-1,Company - Private,Construction,"Construction, Repair & Maintenance",Unknown / Non-Applicable,-1,-1,3,1,34,53
7033,"Clinical Laboratory Scientist (Per Diem, Varia...",$34-$53 Per Hour(Glassdoor est.),"Monterey Park Hospital, located in the San Gab...",3.0,AHMC Healthcare\n3.0,"Monterey Park, CA","Alhambra, CA",5001 to 10000 employees,1998,Company - Private,Health Care Services & Hospitals,Health Care,$1 to $2 billion (USD),-1,-1,3,1,34,53
7034,Data Engineer,$34-$53 Per Hour(Glassdoor est.),Responsibilities:* Responsible for building an...,-1.0,BlackLine Systems,"Woodland Hills, CA","Markham, Canada",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,-1,3,1,34,53
7035,Clinical Laboratory Scientist - Generalist,$34-$53 Per Hour(Glassdoor est.),LGC Labs is the trusted partner in laboratory ...,3.8,LetsGetChecked\n3.8,"Monrovia, CA","New York, NY",51 to 200 employees,2015,Company - Private,Health Care Services & Hospitals,Health Care,Unknown / Non-Applicable,-1,-1,3,1,34,53
7036,"Clinical Laboratory Scientist , Med Tech",$34-$53 Per Hour(Glassdoor est.),Job Overview:\n\n\nCovance is looking for a Cl...,3.4,Covance\n3.4,"Los Angeles, CA","Burlington, NC",10000+ employees,1996,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10 to $25 million (USD),"PRA Health Sciences, PPD, IQVIA",-1,3,1,34,53
7640,Stafford Data Science Tutor Jobs,$10-$26 Per Hour(Glassdoor est.),#Stafford Data Science Tutor Jobs\n\nVarsity T...,4.2,Varsity Tutors\n4.2,"Stafford, TX","Saint Louis, MO",201 to 500 employees,2007,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,-1,-1,3,1,10,26
7641,Spring Data Science Tutor Jobs,$10-$26 Per Hour(Glassdoor est.),#Spring Data Science Tutor Jobs\n\nVarsity Tut...,4.2,Varsity Tutors\n4.2,"Spring, TX","Saint Louis, MO",201 to 500 employees,2007,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,-1,-1,3,1,10,26
7642,Sugar Land Data Science Tutor Jobs,$10-$26 Per Hour(Glassdoor est.),#Sugar Land Data Science Tutor Jobs\n\nVarsity...,4.2,Varsity Tutors\n4.2,"Sugar Land, TX","Saint Louis, MO",201 to 500 employees,2007,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,-1,-1,3,1,10,26


In [ ]:
### company name replace \n
df['Company Name'] = df['Company Name'].str.replace(r'\n\S+', '')

In [ ]:
### Location, HQ split into city and state
df['city'] = df['Location'].str.split(', ', n =1,  expand = True)[0]
df['state'] = df['Location'].str.split(', ', n =1,  expand = True)[1]
# df['HQ_city'] = df['Headquarters'].str.split(', ', n =1,  expand = True)[0]
# df['HQ_state'] = df['Headquarters'].str.split(', ', n =1,  expand = True)[1]

In [ ]:
#change state to full name
state_dict = {"AL":"Alabama", "AK":"Alaska", "AZ":"Arizona", "AR":"Arkansas", "CA":"California", "CO":"Colorado", "CT":"Connecticut", 
          "DC":"Washington DC", "DE":"Delaware", "FL":"Florida", "GA":"Georgia", "HI":"Hawaii", "ID":"Idaho", "IL":"Illinois", 
          "IN":"Indiana", "IA":"Iowa", "KS":"Kansas", "KY":"Kentucky", "LA":"Louisiana", "ME":"Maine", "MD":"Maryland",
          "MA":"Massachusetts", "MI":"Michigan", "MN":"Minnesota", "MS":"Mississippi", "MO":"Missouri", "MT":"Montana",
          "NE":"Nebraska", "NV":"Nevada", "NH":"New Hampshire", "NJ":"New Jersey", "NM":"New Mexico", "NY":"New York", 
          "NC":"North Carolina", "ND":"North Dakota", "OH":"Ohio", "OK":"Oklahoma", "OR":"Oregon", "PA":"Pennsylvania", 
          "RI":"Rhode Island", "SC":"South Carolina", "SD":"South Dakota", "TN":"Tennessee", "TX":"Texas", "UT":"Utah", "VT":"Vermont",
          "VA":"Virginia", "WA":"Washington", "WV":"West Virginia","WI":"Wisconsin", "WY":"Wyoming"}

In [ ]:
df['state_full'] = 1
for index, i in enumerate(df['state']):
  if i in state_dict:
    df['state_full'][index] = state_dict[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,Position,Hourly_pay,lower_salary,higher_salary,city,state,state_full
0,Business Analyst - Clinical & Logistics Platform,$56K-$102K (Glassdoor est.),Company Overview\n\n\nAt Memorial Sloan Ketter...,3.9,Memorial Sloan-Kettering,"New York, NY","New York, NY",10000+ employees,1884,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An...",-1,1,0,56000,102000,New York,NY,New York
1,Business Analyst,$56K-$102K (Glassdoor est.),We are seeking for an energetic and collaborat...,3.8,Paine Schwartz Partners,"New York, NY","New York, NY",1 to 50 employees,-1,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1,TRUE,1,0,56000,102000,New York,NY,New York
2,Data Analyst,$56K-$102K (Glassdoor est.),"For more than a decade, Asembia has been worki...",3.6,Asembia,"Florham Park, NJ","Florham Park, NJ",501 to 1000 employees,2004,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$5 to $10 million (USD),-1,-1,1,0,56000,102000,Florham Park,NJ,New Jersey
3,"Information Security Analyst, Incident Response",$56K-$102K (Glassdoor est.),Job Description Summary\nThe Information Secur...,3.6,BD,"Franklin Lakes, NJ","Franklin Lakes, NJ",10000+ employees,1897,Company - Public,Health Care Products Manufacturing,Manufacturing,$10+ billion (USD),"Abbott, Siemens, Baxter",-1,1,0,56000,102000,Franklin Lakes,NJ,New Jersey
4,Analyst - FP&A Global Revenue,$56K-$102K (Glassdoor est.),Magnite is the world's largest independent sel...,3.4,Rubicon Project,"New York, NY","Los Angeles, CA",201 to 500 employees,2007,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"PubMatic, AppNexus, Index Exchange",TRUE,1,0,56000,102000,New York,NY,New York


In [ ]:
#filter rows
df.drop(axis = 1, index = df.loc[df['Hourly_pay'] == 1].index, inplace=True)

#filter columns
df.drop('Headquarters', axis=1, inplace=True)
df.drop('Job Description', axis=1, inplace=True)
df.drop('Competitors', axis=1, inplace=True)
df.drop('Easy Apply', axis=1, inplace=True)
df.drop('Industry', axis=1, inplace=True)
df.drop('Hourly_pay', axis=1, inplace=True)
df.drop('Revenue', axis=1, inplace=True)
df.drop('Location', axis=1, inplace=True)
df.drop('Salary Estimate', axis=1, inplace=True)

In [ ]:
df.drop(axis = 1, index = df.loc[df['Sector'] == '-1'].index, inplace=True)

In [ ]:
df.drop(axis = 1, index = df.loc[df['Size'] == '-1'].index, inplace=True)

In [ ]:
df.drop(axis = 1, index = df.loc[df['Size'] == 'Unknown'].index, inplace=True)

In [ ]:
df.drop(axis = 1, index = df.loc[df['higher_salary'] == '1'].index, inplace=True)

In [ ]:
df.drop(axis = 1, index = df.loc[df['state_full'] == 1].index, inplace=True)

In [ ]:
df.drop(axis = 1, index = df.loc[df['Type of ownership'] == 'Unknown'].index, inplace=True)

In [ ]:
df.drop(axis = 1, index = df.loc[df['Founded'] == -1].index, inplace=True)

In [ ]:
df.head()

,Job Title,Rating,Company Name,Size,Founded,Type of ownership,Sector,Position,lower_salary,higher_salary,city,state,state_full
0,Business Analyst - Clinical & Logistics Platform,3.9,Memorial Sloan-Kettering,10000+ employees,1884,Nonprofit Organization,Health Care,1,56000,102000,New York,NY,New York
2,Data Analyst,3.6,Asembia,501 to 1000 employees,2004,Company - Private,Biotech & Pharmaceuticals,1,56000,102000,Florham Park,NJ,New Jersey
3,"Information Security Analyst, Incident Response",3.6,BD,10000+ employees,1897,Company - Public,Manufacturing,1,56000,102000,Franklin Lakes,NJ,New Jersey
4,Analyst - FP&A Global Revenue,3.4,Rubicon Project,201 to 500 employees,2007,Company - Public,Information Technology,1,56000,102000,New York,NY,New York
5,Data Analyst,3.4,Sapphire Digital,201 to 500 employees,2019,Company - Private,Information Technology,1,56000,102000,Lyndhurst,NJ,New Jersey


In [ ]:
df.reset_index(inplace = True ,drop= True)

In [ ]:
a = df['state'].unique()
print(a)

['NY' 'NJ' 'CA' 'IL' 'TX' 'AZ' 'PA' 'DE' 'FL' 'UT' 'VA' 'OH' 'NC' 'SC'
 'IN' 'WA' 'GA' 'KS' 'CO']


In [ ]:
# from google.colab import files

# df.to_csv('df.csv')
# files.download('df.csv')